In [1]:
# from fdh_gallica import Periodical, Search, Document
# from fdh_gallica.parallel_process import iiif_urls_for_documents
from tqdm.autonotebook import tqdm
import json
import pandas as pd
from pandas.io.json import json_normalize
import itertools
from itertools import chain
from itertools import compress
import re
import wikipedia
from bs4 import BeautifulSoup
import requests
# from SPARQLWrapper import SPARQLWrapper, XML
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
bnf_table = pd.read_pickle('data/bnf_table_summaries.pkl')

In [3]:

bnf_note_ = list(chain.from_iterable(list(map(lambda x: re.sub(' et ',',',x).split(','),\
                                             bnf_table.note.dropna().unique().tolist()))))
#bnf_note = list(chain.from_iterable(list(map(lambda x: re.sub('[()\d\.\-:,"]','',x).split(),bnf_note_))))

In [4]:
bnf_note = pd.Series(bnf_note_)

In [5]:
bnf_note = bnf_note.apply(lambda x: re.sub("\(.*\d+.*\)",'',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[\(\)]",' ',x))


In [6]:
bnf_note = bnf_note.apply(lambda x: re.sub("[Mm]ezzo[\s-]soprano",'mezzosoprano',x))
#bnf_note = bnf_note.apply(lambda x: re.sub("[Mm]ezzo-soprano",'mezzosoprano',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Bb]aryton[\s-]basse",'barytonbasse',x))
#bnf_note = bnf_note.apply(lambda x: re.sub("[Bb]aryton-basse",'barytonbasse',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Hh]omme[\s-][Pp]olitique",'hommepolitique',x))
#bnf_note = bnf_note.apply(lambda x: re.sub("[Hh]omme [Pp]olitique",'hommepolitique',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Cc]hef d'[Oo]rchestre",'chefdorchetre',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Hh]omme de [Ll]ettres",'hommedelettres',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Aa]rtiste [Ll]yrique",'artistelyrique',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Aa]cadémie de [Mm]édecine",'academiedemédecine',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Aa]cadémie des [Sc]iences",'academiedesciences',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Aa]cadémie [Ff]rançaise",'academiefrançaise',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Aa]cadémie [Dd]es [Bb]eaux-arts",'academiedesbeaxarts',x))
bnf_note = bnf_note.apply(lambda x: re.sub("[Ll]égion d'[Hh]onneur",'legiondhonneur',x))

In [7]:
bnf_note = bnf_note.apply(lambda x: re.sub("['\.-]",' ',x))

In [8]:

namelist_ = bnf_table.name.apply(lambda x: re.split("[ ,'-]", re.sub('\(.*\)','',x))).tolist()
nameset = list(chain.from_iterable(namelist_))
nameset = list(map(lambda x:x.strip(),namelist))
nameset = set(namelist) - set([''])

NameError: name 'namelist' is not defined

In [12]:
bnf_word_ = list(chain.from_iterable(list(map(lambda x: x.split(),bnf_note.tolist()))))
bnf_word_ = list(map(lambda x:x.lower(),bnf_word_))
bnf_word = pd.Series(bnf_word_)


In [686]:
#bnf_note[bnf_note.apply(lambda x: "cinéma" in x.lower())].tolist()

In [647]:
bnf_word_count = bnf_word.value_counts()
bnf_word_count = bnf_word_count.reset_index().rename(columns={'index':'word'})
bnf_word_count['strlen'] = bnf_word_count.word.apply(lambda x: len(x))

In [674]:
taglist = bnf_word_count.word[(bnf_word_count.word.apply(lambda x: x not in nameset)) & \
               (bnf_word_count.strlen > 4) & (bnf_word_count[0] > 4)].tolist()

In [681]:
taglist = ['academiedemédecine', 'academiedesbeaxarts','academiefrançaise',
            'acteur', 'actrice', 'administrateur', 'agriculture', 'agronome', 'ambassadeur', 'anarchiste',
            'aquarelliste', 'architecte', 'archiviste', 'artiste', 'artistelyrique', 'auteur', 'avocat',
 'aéronaute', 'banquier', 'baptiste', 'baron', 'baryton', 'barytonbasse', 'basse', 'bibliothécaire',
 'bouffes', 'cantatrice', 'capitaine', 'caricaturiste', 'carrière', 'chansonnier', 'chanteur', 'chanteuse',
 'chefdorchetre', 'chevalier', 'chimiste', 'chirurgien', 'chroniqueur', 'cinéma', 'collaborateur',
 'collectionneur', 'collège', 'colonel', 'comique', 'commandant', 'commissaire', 'compositeur', 'comédie',
 'comédien', 'comédienne', 'conseiller', 'conservateur', 'conteur', 'critique', 'créateur', 'créatrice',
 'danseuse', 'dessinateur', 'diplomate', 'directeur', 'directrice', 'docteur', 'dramatique', 'dramaturge',
 'droit', 'décorateur', 'député', 'ecrivain', 'empire', 'enseignement', 'essayiste', 'explorateur', 'faculté',
 'finance', 'fondateur', 'fondatrice', 'frère', 'gouvernement', 'gouverneur', 'graveur', 'gymnase', 'général',
 'géographie', 'historien', 'hommedelettres', 'hommepolitique', 'illustrateur', 'imprimeur', 'ingénieur',
 'inspecteur', 'inventeur', 'journaliste', 'legiondhonneur', 'lettres', 'libraire', 'librettiste', 'lieutenant',
 'lithographe', 'lyrique', 'magistrat', 'maire', 'marine', 'maître', 'metteur', 'mezzosoprano', 'militaire',
 'militant', 'ministre', 'ministère', 'municipal', 'musical', 'musique', 'médecin', 'médecine', 'médicale',
 'observatoire', 'officier', 'organiste', 'orientaliste', 'parolier', 'pastelliste', 'pasteur', 'patronyme',
 'paysagiste', 'peintre', 'pensionnaire', 'perpétuel', 'philosophe', 'photographe', 'physicien', 'physiologie',
 'pianiste', 'polytechnique', 'poète', 'prince', 'professeur', 'propriétaire', 'préfet',
 'président', 'prêtre', 'publiciste', 'romancier', 'romancière', 'rédacteur', 'républicain', 'scientifique',
 'sculpteur', 'secrétaire', 'sociétaire', 'soprano', 'spécialiste', 'sénateur', 'théophile', 'théoricien',
 'théâtre', 'titulaire', 'traducteur', 'ténor', 'vaudeville', 'violoniste', 'voyageur', 'économiste',
 'écrivain', 'éditeur' ]


In [747]:
taglist.sort(key=lambda s: len(s))
taglist

['baron',
 'basse',
 'droit',
 'frère',
 'maire',
 'poète',
 'ténor',
 'acteur',
 'auteur',
 'avocat',
 'cinéma',
 'député',
 'empire',
 'marine',
 'maître',
 'prince',
 'préfet',
 'prêtre',
 'actrice',
 'artiste',
 'baryton',
 'bouffes',
 'collège',
 'colonel',
 'comique',
 'comédie',
 'conteur',
 'docteur',
 'faculté',
 'finance',
 'graveur',
 'gymnase',
 'général',
 'lettres',
 'lyrique',
 'metteur',
 'musical',
 'musique',
 'médecin',
 'pasteur',
 'peintre',
 'soprano',
 'théâtre',
 'éditeur',
 'agronome',
 'banquier',
 'baptiste',
 'carrière',
 'chanteur',
 'chimiste',
 'comédien',
 'critique',
 'créateur',
 'danseuse',
 'ecrivain',
 'libraire',
 'militant',
 'ministre',
 'médecine',
 'médicale',
 'officier',
 'parolier',
 'pianiste',
 'sénateur',
 'voyageur',
 'écrivain',
 'aéronaute',
 'capitaine',
 'chanteuse',
 'chevalier',
 'créatrice',
 'diplomate',
 'directeur',
 'essayiste',
 'fondateur',
 'historien',
 'imprimeur',
 'ingénieur',
 'inventeur',
 'magistrat',
 'militaire',
 

In [765]:
def get_tag(x):
    
    #exceptions
    res = []
    try:
        x = re.sub("[Mm]ezzo[\s-]soprano",'mezzosoprano',x)
        x = re.sub("[Bb]aryton[\s-]basse",'barytonbasse',x)
        x = re.sub("[Hh]omme[\s-][Pp]olitique",'hommepolitique',x)
        x = re.sub("[Cc]hef d'[Oo]rchestre",'chefdorchetre',x)
        x = re.sub("[Hh]omme de [Ll]ettres",'hommedelettres',x)
        x = re.sub("[Aa]rtiste [Ll]yrique",'artistelyrique',x)
        x = re.sub("[Aa]cadémie de [Mm]édecine",'academiedemédecine',x)
        x =  re.sub("[Aa]cadémie des [Sc]iences",'academiedesciences',x)
        x =  re.sub("[Aa]cadémie [Ff]rançaise",'academiefrançaise',x)
        x = re.sub("[Aa]cadémie [Dd]es [Bb]eaux-arts",'academiedesbeaxarts',x)
        x = re.sub("[Ll]égion d'[Hh]onneur",'legiondhonneur',x)
        x = re.sub("['\.-]",' ',x)
        x = x.lower()
        for t in taglist:
            if t in x:
                res.append(t)
    except:
        res = []
    
    # remove duplicates
    remove = []
    for i in res:
        temp = list(compress(res,list(map(lambda x: i in x,res))))
        if len(temp) != 1:
            remove.append(i)
                    
    
    return list(set(res) - set(remove))

In [766]:
def country_clean(x):
    try:
        if x == 'non renseigné':
            return []
        else:    
            x = x.split('avant')[0].strip()
            x = x.split('(')[0].strip()
            return [x]
    except:
        return []

In [767]:
bnf_table['tags'] = bnf_table.note.apply(get_tag) + bnf_table.country.apply(country_clean)

In [768]:
bnf_table.tags

0                        [député, hommepolitique, France]
1                                               [soprano]
2                  [peintre, écrivain, sculpteur, France]
3                [chimiste, photographe, Grande-Bretagne]
4                   [soprano, artistelyrique, États-Unis]
                              ...                        
1926    [journaliste, avocat, lettres, docteur, romanc...
1927                                  [diplomate, France]
1928    [peintre, graveur, architecte, sculpteur, France]
1929           [metteur, cinéma, théâtre, acteur, France]
1930                                 [professeur, Russie]
Name: tags, Length: 1874, dtype: object

In [698]:
bnf_table[['name','tags']].to_pickle('data/bnf_tags.pkl')